In [17]:
!kubecolor -n jfrog get all

NAME                          READY   STATUS    RESTARTS   AGE
pod/jfrog-artifactory-jcr-0   4/6     Running   0          38s

NAME                            TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                      AGE
service/jfrog-artifactory-jcr   ClusterIP   10.108.45.209   <none>        8082/TCP,8025/TCP,8081/TCP   43s

NAME                                     READY   AGE
statefulset.apps/jfrog-artifactory-jcr   0/1     43s


In [18]:
!kubecolor -n jfrog get services

NAME                    TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                      AGE
jfrog-artifactory-jcr   ClusterIP   10.108.45.209   <none>        8082/TCP,8025/TCP,8081/TCP   45s


In [19]:
!kubectl  -n jfrog get services jfrog-artifactory-jcr -o yaml | yq

apiVersion: v1
kind: Service
metadata:
  annotations:
    meta.helm.sh/release-name: jfrog-artifactory-jcr
    meta.helm.sh/release-namespace: jfrog
  creationTimestamp: "2024-02-05T14:30:25Z"
  labels:
    app: artifactory
    app.kubernetes.io/managed-by: Helm
    chart: artifactory-107.77.3
    component: artifactory
    heritage: Helm
    release: jfrog-artifactory-jcr
  name: jfrog-artifactory-jcr
  namespace: jfrog
  resourceVersion: "2856363"
  uid: c526b647-458d-49f4-98fc-c5bc65585f89
spec:
  clusterIP: 10.108.45.209
  clusterIPs:
    - 10.108.45.209
  internalTrafficPolicy: Cluster
  ipFamilies:
    - IPv4
  ipFamilyPolicy: SingleStack
  ports:
    - name: http-router
      port: 8082
      protocol: TCP
      targetPort: 8082
    - name: http-rtfs
      port: 8025
      protocol: TCP
      targetPort: 8025
    - name: http-artifactory
      port: 8081
      protocol: TCP
      targetPort: 8081
  selector:
    app: artifactory
    component: artifactory
    release: jfrog-arti

In [23]:
!nmap -p 8081,8082,8025 jfrog-artifactory-jcr.jfrog.svc

Starting Nmap 7.80 ( https://nmap.org ) at 2024-02-05 11:32 -03
Nmap scan report for jfrog-artifactory-jcr.jfrog.svc (10.108.45.209)
Host is up (0.00019s latency).
rDNS record for 10.108.45.209: jfrog-artifactory-jcr.jfrog.svc.cluster2.xpt

PORT     STATE  SERVICE
8025/tcp closed ca-audit-da
8081/tcp open   blackice-icecap
8082/tcp open   blackice-alerts

Nmap done: 1 IP address (1 host up) scanned in 0.12 seconds


In [32]:
!dig lab.jfrog.worldl.xpt


; <<>> DiG 9.18.18-0ubuntu0.22.04.1-Ubuntu <<>> lab.jfrog.worldl.xpt
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 1985
;; flags: qr rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 1232
; COOKIE: 0648428ef576775e (echoed)
;; QUESTION SECTION:
;lab.jfrog.worldl.xpt.		IN	A

;; ANSWER SECTION:
lab.jfrog.worldl.xpt.	30	IN	A	192.168.49.2

;; Query time: 3 msec
;; SERVER: 10.96.0.10#53(10.96.0.10) (UDP)
;; WHEN: Mon Feb 05 15:47:59 -03 2024
;; MSG SIZE  rcvd: 97



In [27]:
!kubecolor -n jfrog get ingress

NAME                    CLASS   HOSTS                  ADDRESS        PORTS   AGE
jfrog-artifactory-jcr   nginx   jfrog-jcr.worldl.xpt   192.168.49.2   80      3h54m
jfrog-ingress-lab       nginx   lab.jfrog.worldl.xpt   192.168.49.2   80      3m16s


   * http://lab.jfrog.worldl.xpt/ui/repos/tree/Properties/lab

   * http://jfrog-jcr.worldl.xpt
      * service "jfrog-artifactory-jcr" port 8082
   * http://jfrog-jcr.worldl.xpt/artifactory
      * service "jfrog-artifactory-jcr" port 8081

In [35]:
!kubectl -n jfrog get ingress jfrog-artifactory-jcr -o yaml | yq

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    meta.helm.sh/release-name: jfrog-artifactory-jcr
    meta.helm.sh/release-namespace: jfrog
  creationTimestamp: "2024-02-05T14:30:25Z"
  generation: 1
  labels:
    app: artifactory
    app.kubernetes.io/managed-by: Helm
    chart: artifactory-107.77.3
    heritage: Helm
    release: jfrog-artifactory-jcr
  name: jfrog-artifactory-jcr
  namespace: jfrog
  resourceVersion: "2856550"
  uid: b648eb50-ddcb-4b89-92db-50834c448f82
spec:
  defaultBackend:
    service:
      name: jfrog-artifactory-jcr
      port:
        number: 8082
  ingressClassName: nginx
  rules:
    - host: jfrog-jcr.worldl.xpt
      http:
        paths:
          - backend:
              service:
                name: jfrog-artifactory-jcr
                port:
                  number: 8082
            path: /
            pathType: ImplementationSpecific
          - backend:
              service:
                name: jfrog-artifactory-jcr
  

In [36]:
!kubectl -n jfrog get ingress jfrog-ingress-lab -o yaml | yq

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"networking.k8s.io/v1","kind":"Ingress","metadata":{"annotations":{"nginx.ingress.kubernetes.io/backend-protocol":"HTTP"},"name":"jfrog-ingress-lab","namespace":"jfrog"},"spec":{"ingressClassName":"nginx","rules":[{"host":"lab.jfrog.worldl.xpt","http":{"paths":[{"backend":{"service":{"name":"jfrog-artifactory-jcr-artifactory-nginx","port":{"number":80}}},"path":"/","pathType":"Prefix"}]}}]}}
    nginx.ingress.kubernetes.io/backend-protocol: HTTP
  creationTimestamp: "2024-02-05T18:21:55Z"
  generation: 2
  name: jfrog-ingress-lab
  namespace: jfrog
  resourceVersion: "3330217"
  uid: 054d710d-a4f4-49e8-92ff-e57a65e50c67
spec:
  ingressClassName: nginx
  rules:
    - host: lab.jfrog.worldl.xpt
      http:
        paths:
          - backend:
              service:
                name: jfrog-artifactory-jcr-artifactory-nginx
                p

   * <span style="color: blue;">/etc/nginx/conf.d/artifactory.conf</span> 

  
```text
ssl_protocols TLSv1 TLSv1.1 TLSv1.2 TLSv1.3;
ssl_certificate  /var/opt/jfrog/nginx/ssl/tls.crt;
ssl_certificate_key  /var/opt/jfrog/nginx/ssl/tls.key;
ssl_session_cache shared:SSL:1m;
ssl_prefer_server_ciphers   on;
## server configuration
server {
  listen 8443 ssl;
  listen 8080;
  server_name ~(?<repo>.+)\.jfrog-artifactory-jcr jfrog-artifactory-jcr ~(?<repo>.+)\.jfrog-jcr.worldl.xpt;

  if ($http_x_forwarded_proto = '') {
    set $http_x_forwarded_proto  $scheme;
  }
  ## Application specific logs
  ## access_log /var/log/nginx/artifactory-access.log timing;
  ## error_log /var/log/nginx/artifactory-error.log;
  rewrite ^/artifactory/?$ / redirect;
  if ( $repo != "" ) {
    rewrite ^/(v1|v2)/(.*) /artifactory/api/docker/$repo/$1/$2 break;
  }
  chunked_transfer_encoding on;
  client_max_body_size 0;

  location / {
    proxy_read_timeout  900;
    proxy_pass_header   Server;
    proxy_cookie_path   ~*^/.* /;
    proxy_pass          http://jfrog-artifactory-jcr:8082/;
    proxy_set_header    X-JFrog-Override-Base-Url $http_x_forwarded_proto://$host:$server_port;
    proxy_set_header    X-Forwarded-Port  $server_port;
    proxy_set_header    X-Forwarded-Proto $http_x_forwarded_proto;
    proxy_set_header    Host              $http_host;
    proxy_set_header    X-Forwarded-For   $proxy_add_x_forwarded_for;
    add_header Strict-Transport-Security "max-age=31536000; includeSubDomains" always;

    location /artifactory/ {
      if ( $request_uri ~ ^/artifactory/(.*)$ ) {
        proxy_pass       http://jfrog-artifactory-jcr:8081/artifactory/$1;
      }
      proxy_pass         http://jfrog-artifactory-jcr:8081/artifactory/;
    }
    location /pipelines/ {
      proxy_http_version 1.1;
      proxy_set_header Upgrade $http_upgrade;
      proxy_set_header Connection "upgrade";
      proxy_set_header Host $http_host;
      proxy_pass  http://jfrog-artifactory-jcr:8082;
    }
  }
}
```


   * https://jfrog.worldl.xpt/artifactory/lab/